In [1]:
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv

load_dotenv()

#### SerperDevTool와 WebsiteSearchTool를 활용해 태크뉴스를 작성하는 Task를 만들어보자

In [2]:
from crewai import Agent, Task, Crew
from crewai.process import Process
# Importing crewAI tools

# 검색 도구에서 가장 대표적인 3가지 도구
from crewai_tools import (
    SerperDevTool,  # Serper API를 활용한 결과물을 받을 수 있는 도구. 특정 키워드를 주면, 그걸 구글에 검색한 결과를 가져옴.
    WebsiteSearchTool, # 특정 키워드 그리고 어떤 웹사이트를 주면 해당 웹사이트 내에서 해당 키워드와 관련이 있는 부분만 가져옴. RAG가 내부적으로 실행이 되는 도구라고 보면 된다. 웹사이트 전체를 긁어오는게 아닌 키워드와 관련이 있는 부분만 가져옴
    ScrapeWebsiteTool # 특정 웹사이트를 주면 해당 웹사이트 내에서 모든 텍스트를 스크래핑해서 가져옴. (크롤링 같은거)
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini") # gpt-4o-mini 모델을 사용

search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

# 테크 뉴스레터를 작성하는 에이전트 만들기

# 테크 트렌드 연구원
researcher = Agent(
    role='테크 트렌드 연구원',
    goal="인공지능 분야의 최신 기술 트렌드를 한국어로 제공합니다. 지금은 2024년 9월입니다.", # 최신 정보를 가져올 수 있게 2024년 9월 이라고 현재 시점을 명시적이로 작성
    backstory='기술 트렌드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.', # 어떤 문장을 넣어도 상관없지만, role과 goal에서 설명하지 못한 추가적인 내용들을 담아주면 좋다. (길게 작성해도 되고ㅡ )
    tools=[search_tool, web_rag_tool], # tool은 검색 도구과, 웹사이트 검색 도구를 사용.
    verbose=True, # 이 에이전트가 제대로 수행하고 있는지 결과물을 확인할 수 있도록 출력
    max_iter=5, # (최대 5번만 실행) 해당 에이전트가 주어진 테스크를 수행하는데에 있어서 반복수행하는 것을 제한하는 것 (max_iter값을 정해주지 않으면 goal에서 명령한 값을을 계속 수행함, 즉 웹 검색을 계속 반복해서 토큰수가 엄청 많이 나감, 그래서 최대 웹 검색 결과를 5번만 받아오도록 제한)
    llm = llm # 
)

# 주어진 테크 트렌드 연구원의 정보를 기반으로 해서 작성하는 뉴스레터 에이전트
writer = Agent(
    role='뉴스레터 작성자',
    goal="최신 AI 기술 트렌드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2024년 8월입니다.",
    backstory='기술에 대한 열정을 가진 숙련된 작가입니다.',
    verbose=True,
    allow_delegation=False, # 다른 에이전트에게 작업을 위임하는 것을 허용하지 않음 (writer에이전트만 수행하게 만듬) 이 부분을 설정하지 않으면 가끔 가다 writer에이전트가 뉴스레터를 작성하는 과정에서 부족한 정보를 researcher에게 계속해서 요청하는 모습을 볼 수 있음.
    llm = llm
)

# Define tasks
# 각 각의 에이전트(researcher, writer)가 수행해야하는 테스크 정의

# 첫 번째로 researcher 에이전트가 수행해야하는 테스크 정의
research = Task(
    description='AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.',
    expected_output='AI 업계에서 가장 주목받는 3대 기술 개발 동향과 그 중요성에 대한 신선한 관점을 요약한 글',
    agent=researcher # 이 테스크를 researcher 에이전트가 수행하도록 지정
)

# 두 번째로 writer 에이전트가 수행해야하는 테스크 정의
write = Task(
    description="""테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
    테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.""", # 테크 뉴스레터 이므로 특별히 전문적인 용어 사용 허용
    expected_output='최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터', # 결과물을 재밌는 말투로 4문단 짜리 마크다운 형싱으로 작성
    agent=writer, # 이 테스크를 writer 에이전트가 수행하도록 지정
    output_file=r'C:\Users\user\new_post.md'  # 주어진 경로에다가 해당 테스크의 결과물을 저장
)

# Assemble a crew with planning enabled
# 에이전트들과 테스크를 조합해서 크루 만들기
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    process=Process.sequential # 테스크를 순차적으로 수행하도록 지정
)

# Execute tasks
# 크루 실행
result = crew.kickoff()

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))